In [2]:
from __future__ import division
import math, random, re
from collections import defaultdict, Counter, deque
from functools import partial

def vector_add(v, w):
    """adds two vectors componentwise"""
    return [v_i + w_i for v_i, w_i in zip(v,w)]

def vector_subtract(v, w):
    """subtracts two vectors componentwise"""
    return [v_i - w_i for v_i, w_i in zip(v,w)]

def vector_sum(vectors):
    return reduce(vector_add, vectors)

def scalar_multiply(c, v):
    return [c * v_i for v_i in v]

# this isn't right if you don't from __future__ import division
def vector_mean(vectors):
    """compute the vector whose i-th element is the mean of the
    i-th elements of the input vectors"""
    n = len(vectors)
    return scalar_multiply(1/n, vector_sum(vectors))

def dot(v, w):
    """v_1 * w_1 + ... + v_n * w_n"""
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

def sum_of_squares(v):
    """v_1 * v_1 + ... + v_n * v_n"""
    return dot(v, v)

def magnitude(v):
    return math.sqrt(sum_of_squares(v))

def squared_distance(v, w):
    return sum_of_squares(vector_subtract(v, w))

def distance(v, w):
   return math.sqrt(squared_distance(v, w))

#
# functions for working with matrices
#

def shape(A):
    num_rows = len(A)
    num_cols = len(A[0]) if A else 0
    return num_rows, num_cols

def get_row(A, i):
    return A[i]
    
def get_column(A, j):
    return [A_i[j] for A_i in A]

def make_matrix(num_rows, num_cols, entry_fn):
    """returns a num_rows x num_cols matrix 
    whose (i,j)-th entry is entry_fn(i, j)"""
    return [[entry_fn(i, j) for j in range(num_cols)]
            for i in range(num_rows)]  




In [3]:
users = [
    { "id": 0, "name": "Hero" },
    { "id": 1, "name": "Dunn" },
    { "id": 2, "name": "Sue" },
    { "id": 3, "name": "Chi" },
    { "id": 4, "name": "Thor" },
    { "id": 5, "name": "Clive" },
    { "id": 6, "name": "Hicks" },
    { "id": 7, "name": "Devin" },
    { "id": 8, "name": "Kate" },
    { "id": 9, "name": "Klein" }
]

friendships = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
               (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]


In [4]:
# give each user a friends list
for user in users:
    user["friends"] = []
    
# and populate it
for i, j in friendships:
    # this works because users[i] is the user whose id is i
    users[i]["friends"].append(users[j]) # add i as a friend of j
    users[j]["friends"].append(users[i]) # add j as a friend of i   



In [5]:
from collections import deque

def shortest_paths_from(from_user):

    # a dictionary from "user_id" to *all* shortest paths to that user
    shortest_paths_to = { from_user["id"] : [[]] }

    # a queue of (previous user, next user) that we need to check.
    # starts out with all pairs (from_user, friend_of_from_user)
    frontier = deque((from_user, friend)
                     for friend in from_user["friends"])

    # keep going until we empty the queue
    while frontier:

        prev_user, user = frontier.popleft()   # remove the user who's
        user_id = user["id"]                   # first in the queue

        # because of the way we're adding to the queue,
        # necessarily we already know some shortest paths to prev_user
        paths_to_prev_user = shortest_paths_to[prev_user["id"]]
        new_paths_to_user = [path + [user_id] for path in paths_to_prev_user]

        # it's possible we already know a shortest path
        old_paths_to_user = shortest_paths_to.get(user_id, [])

        # what's the shortest path to here that we've seen so far?
        if old_paths_to_user:
            min_path_length = len(old_paths_to_user[0])
        else:
            min_path_length = float('inf')

        # only keep paths that aren't too long and are actually new
        new_paths_to_user = [path
                             for path in new_paths_to_user
                             if len(path) <= min_path_length
                             and path not in old_paths_to_user]

        shortest_paths_to[user_id] = old_paths_to_user + new_paths_to_user

        # add never-seen neighbors to the frontier
        frontier.extend((user, friend)
                        for friend in user["friends"]
                        if friend["id"] not in shortest_paths_to)

    return shortest_paths_to

In [7]:
[[1,2],[5]]+[[7,8,9]]

[[1, 2], [5], [7, 8, 9]]

In [8]:
a=[1,2,3]
a.extend([5])

In [9]:
a

[1, 2, 3, 5]

In [10]:
for user in users:
    user["shortest_paths"] = shortest_paths_from(user)

In [12]:
user["shortest_paths"]

{0: [[8, 6, 5, 4, 3, 1, 0],
  [8, 7, 5, 4, 3, 1, 0],
  [8, 6, 5, 4, 3, 2, 0],
  [8, 7, 5, 4, 3, 2, 0]],
 1: [[8, 6, 5, 4, 3, 1], [8, 7, 5, 4, 3, 1]],
 2: [[8, 6, 5, 4, 3, 2], [8, 7, 5, 4, 3, 2]],
 3: [[8, 6, 5, 4, 3], [8, 7, 5, 4, 3]],
 4: [[8, 6, 5, 4], [8, 7, 5, 4]],
 5: [[8, 6, 5], [8, 7, 5]],
 6: [[8, 6]],
 7: [[8, 7]],
 8: [[8]],
 9: [[]]}

In [13]:
for user in users:
    user["betweenness_centrality"] = 0.0

for source in users:
    source_id = source["id"]
    for target_id, paths in source["shortest_paths"].iteritems():
        if source_id < target_id:      # don't double count
            num_paths = len(paths)     # how many shortest paths?
            contrib = 1 / num_paths    # contribution to centrality
            for path in paths:
                for id in path:
                    if id not in [source_id, target_id]:
                        users[id]["betweenness_centrality"] += contrib

In [15]:
[ (user["id"], user["betweenness_centrality"]) for user in users]

[(0, 0.0),
 (1, 3.5),
 (2, 3.5),
 (3, 18.0),
 (4, 20.0),
 (5, 20.5),
 (6, 6.0),
 (7, 6.0),
 (8, 8.5),
 (9, 0.0)]

In [16]:
def farness(user):
    """the sum of the lengths of the shortest paths to each other user"""
    return sum(len(paths[0])
               for paths in user["shortest_paths"].values())

In [17]:
for user in users:
    user["closeness_centrality"] = 1 / farness(user)

In [18]:
[ (user["id"], user["closeness_centrality"]) for user in users]

[(0, 0.029411764705882353),
 (1, 0.037037037037037035),
 (2, 0.037037037037037035),
 (3, 0.045454545454545456),
 (4, 0.05),
 (5, 0.05),
 (6, 0.041666666666666664),
 (7, 0.041666666666666664),
 (8, 0.03571428571428571),
 (9, 0.027777777777777776)]

### Eigenvector Centrality

In [19]:
def matrix_product_entry(A, B, i, j):
    return dot(get_row(A, i), get_column(B, j))

In [20]:
def matrix_multiply(A, B):
    n1, k1 = shape(A)
    n2, k2 = shape(B)
    if k1 != n2:
        raise ArithmeticError("incompatible shapes!")

    return make_matrix(n1, k2, partial(matrix_product_entry, A, B))

In [21]:
v = [1, 2, 3]
v_as_matrix = [[1],
               [2],
               [3]]

In [29]:
def vector_as_matrix(v):
    """returns the vector v (represented as a list) as a n x 1 matrix"""
    return [[v_i] for v_i in v]

def vector_from_matrix(v_as_matrix):
    """returns the n x 1 matrix as a list of values"""
    return [row[0] for row in v_as_matrix]

def matrix_operate(A, v):
    v_as_matrix = vector_as_matrix(v)
    product = matrix_multiply(A, v_as_matrix)
    return vector_from_matrix(product)

In [30]:
def find_eigenvector(A, tolerance=0.00001):
    guess = [random.random() for __ in A]

    while True:
        result = matrix_operate(A, guess)
        length = magnitude(result)
        next_guess = scalar_multiply(1/length, result)

        if distance(guess, next_guess) < tolerance:
            return next_guess, length   # eigenvector, eigenvalue

        guess = next_guess

In [31]:
def entry_fn(i, j):
    return 1 if (i, j) in friendships or (j, i) in friendships else 0

n = len(users)
adjacency_matrix = make_matrix(n, n, entry_fn)

In [32]:
adjacency_matrix

[[0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 1, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 1, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]

In [33]:
eigenvector_centralities, _ = find_eigenvector(adjacency_matrix)

In [39]:
eigenvector_centralities

[0.3857804825515394,
 0.5147907609841436,
 0.5147907609841436,
 0.47331228521636876,
 0.2336097841358009,
 0.15014406089926882,
 0.08355433804419286,
 0.08355433804419286,
 0.07283856577173276,
 0.027294010575660575]

In [45]:
i=0
dot(get_row(adjacency_matrix, i), eigenvector_centralities)/2.6688229150815257

0.38578113075622933

In [42]:
matrix_operate(adjacency_matrix, eigenvector_centralities)

[1.0295815219682871,
 1.3738835287520517,
 1.3738835287520517,
 1.263191306104088,
 0.6234563461156376,
 0.40071846022418667,
 0.22298262667100158,
 0.22298262667100158,
 0.1944026866640463,
 0.07283856577173276]

In [43]:
magnitude(matrix_operate(adjacency_matrix, eigenvector_centralities))

2.6688229150815257

### Directed Graphs and PageRank

In [46]:
endorsements = [(0, 1), (1, 0), (0, 2), (2, 0), (1, 2),
                (2, 1), (1, 3), (2, 3), (3, 4), (5, 4),
                (5, 6), (7, 5), (6, 8), (8, 7), (8, 9)]

for user in users:
    user["endorses"] = []       # add one list to track outgoing endorsements
    user["endorsed_by"] = []    # and another to track endorsements

for source_id, target_id in endorsements:
    users[source_id]["endorses"].append(users[target_id])
    users[target_id]["endorsed_by"].append(users[source_id])

In [47]:
endorsements = [(0, 1), (1, 0), (0, 2), (2, 0), (1, 2),
                (2, 1), (1, 3), (2, 3), (3, 4), (5, 4),
                (5, 6), (7, 5), (6, 8), (8, 7), (8, 9)]

for user in users:
    user["endorses"] = []       # add one list to track outgoing endorsements
    user["endorsed_by"] = []    # and another to track endorsements

for source_id, target_id in endorsements:
    users[source_id]["endorses"].append(users[target_id])
    users[target_id]["endorsed_by"].append(users[source_id])

In [48]:
endorsements_by_id = [(user["id"], len(user["endorsed_by"]))
                      for user in users]

endorsements_by_id

[(0, 2),
 (1, 2),
 (2, 2),
 (3, 2),
 (4, 2),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1)]

In [49]:
sorted(endorsements_by_id,
       key=lambda (user_id, num_endorsements): num_endorsements,
       reverse=True)

[(0, 2),
 (1, 2),
 (2, 2),
 (3, 2),
 (4, 2),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1)]

In [76]:
def page_rank(users, damping = 0.9, num_iters = 1000):

    # initially distribute PageRank evenly
    num_users = len(users)
    pr = { user["id"] : 1 / num_users for user in users }

    # this is the small fraction of PageRank
    # that each node gets each iteration
    base_pr = (1 - damping) / num_users

    for __ in range(num_iters):
        next_pr = { user["id"] : base_pr for user in users }
        for user in users:
            # distribute PageRank to outgoing links
            links_pr = pr[user["id"]] * damping
            for endorsee in user["endorses"]:
                next_pr[endorsee["id"]] += links_pr / len(user["endorses"])

        pr = next_pr

    return pr

In [73]:
pr = page_rank(users)

In [74]:
sum(pr.values())

0.3164608168848936

In [75]:
sorted(pr.items(),key= lambda x: x[1],reverse=True)

[(4, 0.05157905022474107),
 (1, 0.03372093023255815),
 (2, 0.03372093023255815),
 (5, 0.031932773109243695),
 (8, 0.031932773109243695),
 (0, 0.03023255813953489),
 (3, 0.03023255813953489),
 (6, 0.02436974789915966),
 (7, 0.02436974789915966),
 (9, 0.02436974789915966)]

In [59]:
pr.items()

[(0, 0.0404553415061296),
 (1, 0.044921190893169885),
 (2, 0.044921190893169885),
 (3, 0.0404553415061296),
 (4, 0.06785083675770529),
 (5, 0.04344422700587085),
 (6, 0.03346379647749512),
 (7, 0.03346379647749512),
 (8, 0.04344422700587085),
 (9, 0.03346379647749512)]